In [11]:
import pandas as pd

# 1) Read in and (optionally) drop that Unnamed index column
df = pd.read_csv("cleanedDataTransactions.csv", index_col=0)

# 2) Choose your user‐id
user_id = '8bca43a5f08644b18e2f21242300332150afb98b'

# 3) Filter & select
user_df = df.loc[
    df['id'] == user_id,    # keep only this user's rows
    ['fecha', 'monto']       # then select just the date and cost
]

print(user_df)


             fecha  monto
270980  2022-01-02  11.62
270981  2022-01-02  45.96
270982  2022-01-03  69.43
270983  2022-01-05  11.73
270984  2022-01-06  11.62
...            ...    ...
271286  2023-01-22  23.10
271287  2023-01-23  21.38
271288  2023-01-25  12.88
271289  2023-01-29  83.51
271290  2023-01-30  11.73

[311 rows x 2 columns]


In [23]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 1) Load & sort your data (parse dates)
df = pd.read_csv('linearRegression.csv', parse_dates=['fecha'])
df = df.sort_values('fecha')

# 2) Compute running balance with a 500 deposit each month
deposit_amount = 1000.0
df['month'] = df['fecha'].dt.to_period('M')

balances = []
prev_balance = 0.0
for month, grp in df.groupby('month'):
    start_balance = prev_balance + deposit_amount
    running = start_balance - grp['monto'].cumsum()
    balances.extend(running)
    prev_balance = running.iloc[-1]
df['balance'] = balances

# 3) Isolate final month and get daily closing balances for days 1–20
last_month = df['month'].iat[-1]
this_month = df[df['month'] == last_month].copy()
this_month['day'] = this_month['fecha'].dt.day

# take the last balance on each day
daily_balance = this_month[this_month['day'] <= 20] \
    .groupby('day')['balance'] \
    .last()

# 4) Fit a simple linear regression day → balance
X = daily_balance.index.values.reshape(-1, 1)   # day numbers 1…20
y = daily_balance.values
model = LinearRegression().fit(X, y)

# 5) Predict balances for days 21–30
future_days = np.arange(21, 31).reshape(-1, 1)
y_pred = model.predict(future_days)

# 6) Build a 30-day DataFrame, fill actual then overwrite with predictions
first_of_month = last_month.to_timestamp()
dates_full = pd.date_range(start=first_of_month, periods=30, freq='D')
df_full = pd.DataFrame({'fecha': dates_full})
df_full['day'] = df_full['fecha'].dt.day

# map real balances and forward-fill missing
df_full['balance'] = df_full['day'].map(daily_balance).ffill()

# overwrite days 21–30 with model predictions
mask = df_full['day'] >= 21
df_full.loc[mask, 'balance'] = y_pred

# 7) Save to CSV
df_full[['fecha', 'balance']].to_csv('forecasted_balances.csv', index=False)
